In [1]:
import pandas as pd
results=pd.read_csv('reviews.csv')

In [169]:
searchfor=['different','same','previous','purchased','past','run','size','dissapointed','small','largeg','quality','fit','unhappy','change']

In [2]:
df = results.drop_duplicates(subset=['Review'])
df=df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
df['Review']=df['Review'].astype(str)

In [3]:
import numpy as np
(df['Review']=='').sum()
test= df[df.Review != '']


In [4]:
df

,ASIN,ProfileName,Stars,StarsText,Title,ReviewDate,ReviewedAt,URL,Review
0,B01CH3AZMU,Amazon Customer,5.0,5.0 out of 5 stars,Love it!!,"March 30, 2019",the United States,https://www.amazon.com/product-reviews/B01CH3AZMU,Love it!!
1,B01CH3AUZW,Thea S.,5.0,5.0 out of 5 stars,Super Comfortable T-short Bra!,"July 30, 2018",the United States,https://www.amazon.com/product-reviews/B01CH3AUZW,I was a former VS user but they never keep the...
2,B01CH3AUZW,Nish,5.0,5.0 out of 5 stars,Comfortable and fits to size if its the cotton...,"June 4, 2020",the United States,https://www.amazon.com/product-reviews/B01CH3AUZW,"I already had this style bra, I was so happy t..."
3,B01CH3AUZW,Barbara S Anderson,5.0,5.0 out of 5 stars,Satisfaction,"June 25, 2021",the United States,https://www.amazon.com/product-reviews/B01CH3AUZW,"Have been wearing this for at least 5 years, i..."
4,B01CH3AUZW,J Noel,5.0,5.0 out of 5 stars,My favorite bra,"April 14, 2019",the United States,https://www.amazon.com/product-reviews/B01CH3AUZW,Best-fitting bra I have. This is my second one...
...,...,...,...,...,...,...,...,...,...
184302,B07NR2Z46H,Vicki L. Bohnhoff,3.0,3.0 out of 5 stars,The elastic around the leg is picky on my skin...,"February 19, 2021",the United States,https://www.amazon.com/product-reviews/B07NR2Z46H,Thought I was ordering something similar to my...
184303,B07NR2Z46H,Kat,5.0,5.0 out of 5 stars,Great fit,"September 3, 2020",the United States,https://www.amazon.com/product-reviews/B07NR2Z46H,Ordered up one size. This is my second time or...
184304,B07NR2Z46H,Hannah Cash,4.0,4.0 out of 5 stars,Getting the Right Panty for the Right Bum,"June 4, 2021",the United States,https://www.amazon.com/product-reviews/B07NR2Z46H,This is a great panty brief. It has just enoug...
184305,B07NR2Z46H,Amazon Customer,5.0,5.0 out of 5 stars,Full coverage,"February 21, 2021",the United States,https://www.amazon.com/product-reviews/B07NR2Z46H,I have tried one and it's really very comforta...


In [172]:
df=df[df['Review'].str.contains('|'.join(searchfor))]

In [17]:
import pyodbc
aoconnection=pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=fotlsqlsrv01.database.windows.net;DATABASE=AOData;UID=bauser;PWD=3ugaspuW')

cursor=aoconnection.cursor()
cursor.execute('Select top 20 * from pkms.oracle_data_ref')
rows=cursor.fetchall()

upc_df= pd.read_sql("Select * from pkms.oracle_data_ref",aoconnection)
upc_df2=upc_df[['ITEM_NUM_NO_REV','UPC','ITEM_STYLE_NUM']]


C:\Users\sa-datascience\azuredatastudio-python\Scripts\VF_reviews\bin\env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
#upc_df2['ITEM_NO_NUM_REV'] = upc_df2['ITEM_NO_NUM_REV'].str.replace(r'.0$', '')

In [18]:
asins_needed=pd.read_excel("ASINlist copy.xlsx")
asins_needed=asins_needed[['ASIN','UPC','Inventory Item Num']]

In [19]:
asins_needed['Inventory Item Num'] = asins_needed['Inventory Item Num'].str.replace(r'.0$', '')

C:\Users\fbate1\AppData\Local\Temp\4\ipykernel_8260\1879451643.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  asins_needed['Inventory Item Num'] = asins_needed['Inventory Item Num'].str.replace(r'.0$', '')


In [20]:
asins_needed=asins_needed.rename(columns={'Inventory Item Num':'ITEM_NUM_NO_REV'})
asins_needed=asins_needed.merge(upc_df2, on='ITEM_NUM_NO_REV',how='left')
asins_needed=asins_needed.drop(['UPC_y'],axis=1)


In [22]:
asins_needed=asins_needed.rename(columns={'UPC_x':'item.product-id'})
final=df.merge(asins_needed, on='ASIN',how='left')

In [25]:
asins_needed.to_csv('asintoStyle.csv')

In [ ]:
asins_needed['']

In [24]:
final.to_excel('unfiltered_reviews.xlsx')

In [173]:
df['filtrev']=df['Review'].str.lower()

In [174]:
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word

df['filtrev']=df['filtrev'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [175]:
df['filtrev']=df['filtrev'].str.replace('[^\w\s]',"")

In [176]:
stop=stopwords.words('english')
df['filtrev']=df['filtrev'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [177]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\fbate1\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [178]:
df['filtrev'] = df['filtrev'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [102]:
from wordcloud import WordCloud
from wordcloud import STOPWORDS

In [103]:
#df.Stars.value_counts()

In [104]:
import seaborn as sns

In [105]:
#sns.countplot(data=df,x='Stars')

In [179]:
reviews1=df
reviews1.dropna(inplace=True)

In [183]:
reviews1.to_csv('filtered_data1.csv')

In [167]:
reviews1['filteredrev']

KeyError: 'filteredrev'

In [180]:
score_1= reviews1[reviews1['Stars']=='1.0']
score_2=reviews1[reviews1['Stars']=='2.0']
score_3=reviews1[reviews1['Stars']=='3.0']
score_4=reviews1[reviews1['Stars']=='4.0']
score_5=reviews1[reviews1['Stars']=='5.0']

In [181]:
reviews_sample=pd.concat([score_1,score_2,score_3,score_4,score_5],axis=0)
reviews_sample.reset_index(drop=True,inplace=True)

In [109]:
import matplotlib.pyplot as plt

In [ ]:
reviews

In [182]:
reviews_str=reviews_sample.filtrev.cat()
#wordcloud=WordCloud(background_color='white').generate(reviews_str)
#plt.figure(figsize=(50,50))
#lt.imshow(wordcloud,interpolation='bilinear')
#plt.axis("off")
#lt.show()

AttributeError: Can only use .cat accessor with a 'category' dtype

In [113]:
reviews_sample.to_csv('wordcloud_text.csv')